In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
import csv
import time

In [2]:
driver = webdriver.Firefox()

In [3]:
driver.get("https://landrecords.karnataka.gov.in/service2/RTC.aspx")

In [4]:
filename = 'D:\Projects\pyproj_webscraping\DTHV.csv'  # Replace with the actual name of your CSV file

In [5]:
with open(filename, 'r', newline='') as f:
    inp_data = list(csv.reader(f))

In [ ]:
for row in inp_data[1:]:
    district = row[0]
    taluk = row[1]
    hobli = row[2]
    village = row[3]
    HSSN = int(row[4])

    #grabbing controls
    elem_district = driver.find_element(By.ID, "ctl00_MainContent_ddlCDistrict")
    elem_district.send_keys(district)
    time.sleep(1)
    elem_taluk = driver.find_element(By.ID, "ctl00_MainContent_ddlCTaluk")
    elem_taluk.send_keys(taluk)
    time.sleep(1)
    elem_hobli = driver.find_element(By.ID, "ctl00_MainContent_ddlCHobli")
    elem_hobli.send_keys(hobli)
    time.sleep(1)
    elem_village = driver.find_element(By.ID, "ctl00_MainContent_ddlCVillage")
    elem_village.send_keys(village)
    time.sleep(1)
    
    #setting the values in different controls

    data = []
    #Survey Number loop starts
    for surveyno in range(1, HSSN+1):
        elem_surveyno = driver.find_element(By.ID, "ctl00_MainContent_txtCSurveyNo")
        elem_surveyno.clear()
        elem_surveyno.send_keys(surveyno)
        time.sleep(1)
        
        #press submit button
        try:
            elem_submit = driver.find_element(By.ID, "ctl00_MainContent_btnCGo")
            time.sleep(2)
            elem_submit.click()
        except:
            elem_submit.send_keys(Keys.RETURN)
        
        time.sleep(1)
    
        #getting a list of available surnocs
        elem_surnoc = driver.find_element(By.ID, "ctl00_MainContent_ddlCSurnocNo")
        surnoc_list = elem_surnoc.text.split("\n")[1:]
        surnoc_list2 = []
        for surnoc in surnoc_list:
            surnoc = surnoc.strip()
            if len(surnoc) > 0:
                surnoc_list2.append(surnoc)
    
        #traversing through each surnoc
        row = []
        for surnoc in surnoc_list2:
            elem_surnoc.send_keys(surnoc)
            time.sleep(1)
            
            elem_hissano = driver.find_element(By.ID, "ctl00_MainContent_ddlCHissaNo")
            hissano_list = elem_hissano.text.split("\n")[1:]
            hissano_list2 = []
            
            for hissano in hissano_list:
                hissano = hissano.strip()
                if len(hissano) > 0:
                    hissano_list2.append(hissano)
                    
            #returning surveyno, surnoc, hissano
            for hissano in hissano_list2:
                row.append(surveyno)
                row.append(surnoc)
                row.append(hissano)

        data.append(row)
    
    
    
    #creating a new csv file with village name to store all survey numbers:
    filename = village+".csv"
    header = ["SurveyNo", "Surnoc", "HissaNo"]
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(header)
        csv_writer.writerow(data)
